# Working With Time Series in Pandas

## Imports

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (16,9)
plt.rcParams["font.size"] = 20

import warnings
warnings.filterwarnings('ignore')

## Converting to DateTime Type

![I have a Date!](https://media2.giphy.com/media/W2heDDJSLC0thGO7Ld/giphy.gif?cid=ecf05e472da7nf00vbh7xp4lqxyszvszjojoc2qw3uthsz4s&rid=giphy.gif&ct=g)

### But how do I get pandas to understand it and work with it

In [ ]:
pd.to_datetime('Jan 1 1970')

### But sometime pandas isn't able to infer the format of our date

In [ ]:
try:
    pd.to_datetime('Jan:1:1970')
except ValueError as e:
    print('ValueError', e)

### So we need to tell pandas what the foramt is

In [ ]:
pd.to_datetime('Jan:1:1970', format='%b:%d:%Y')

#### There are all sorts of format codes and you can find them here: [Python date format codes](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)

### Now what about a whole column of dates?

In [ ]:
url = "https://gist.githubusercontent.com/ryanorsinger/b309f8db19e0ca71b213d4877d835e77/raw/f5841017310e2f4ca070b313529ceec2375336ba/coffee_consumption.csv"
df = pd.read_csv(url)
df

In [ ]:
df.dtypes

In [ ]:
type(df.iloc[1]['date'])

In [ ]:
df.date = pd.to_datetime(df.date)
df.dtypes

## Working with DateTime Series

In [ ]:
df.date

### `.dt` allows us to extract parts of the date

In [ ]:
df.date.dt.year

In [ ]:
df.date.dt.month

In [ ]:
df.date.dt.day

In [ ]:
df.date.dt.day_name()

### Since they are returned as a Series, we can add them back as columns

In [ ]:
df['year'] = df.date.dt.year
df

In [ ]:
df['month'] = df.date.dt.month
df['day'] = df.date.dt.day
df['weekday'] = df.date.dt.day_name()
df.head()

## Working with more complicated data

In [ ]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1kTrAFSrr-xP3REs0Lly0TdV4ekrHahBXLg9r5qKxmV8/edit#gid=0'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

df = pd.read_csv(csv_export_url)
df.columns = df.columns.str.lower()
df

In [ ]:
df.info()

In [ ]:
df.close.plot()

### How do we make pandas time aware so we can utilize the built in functionality for time series analysis?

#### 3 steps

* Convert 'date/time' column to datetime object in Pandas (pd.to_datetime())
* Set the datetime column as Index
* Sort the datetime index

### Convert to datetime

In [ ]:
df.dtypes

In [ ]:
try:
    pd.to_datetime(df.date)
except ValueError as e:
    print('ValueError', e)

#### So what is happening?

Pandas is telling us it is unable to infer what our date format is.

2020-03-13 12-PM is what our date looks like.

It is made up of several parts:

* a 4 digit year: `%Y`
* followed by a hyphen `-`
* a two digit month: `%m`
* followed by a hyphen `-`
* a two digit day: `%d`
* a space ` `
* a 12-hour clock number: `%I`
* a hyphen `-`
* an AM/PM `%p`

With this info we can now build our format string:

`2020-03-13 12-PM`

`%Y-%m-%d %I-%p`

In [ ]:
pd.to_datetime('2020-03-13 12-PM', format='%Y-%m-%d %I-%p')

In [ ]:
df.date = pd.to_datetime(df.date, format='%Y-%m-%d %I-%p')

In [ ]:
df

In [ ]:
df.dtypes

### Now we can set date to the index and sort the index

In [ ]:
df = df.set_index('date').sort_index()
df

In [ ]:
df.close.plot()

### Let's look at the the mean on each day of the week

#### [Pandas datetimeIndex attributes](https://pandas.pydata.org/docs/reference/api/pandas.DatetimeIndex.html)

In [ ]:
df.index.day_name()

In [ ]:
df['weekday_name'] = df.index.day_name()

In [ ]:
df

In [ ]:
df.groupby('weekday_name').mean()

In [ ]:
df.groupby('weekday_name').mean().close.plot()

In [ ]:
df.groupby('weekday_name').mean().volume.plot()

#### The days are out of order so let's fix that

In [ ]:
df.index

In [ ]:
df.index.day_of_week

In [ ]:
df['weekday_number'] = df.index.day_of_week
df

In [ ]:
df.groupby('weekday_name').mean()

In [ ]:
df.groupby('weekday_name').mean().sort_values('weekday_number')

In [ ]:
df.groupby('weekday_name').mean().sort_values('weekday_number').volume.plot()

In [ ]:
df.groupby('weekday_name').mean().sort_values('weekday_number').close.plot()

![Nice gif](https://c.tenor.com/bEBxkuyFiucAAAAC/yes-nice.gif)

### How do we get a subset of the dataframe?

In [ ]:
df.loc['2019-11-19 12:00:00']

In [ ]:
df['2018':'2019']

In [ ]:
df['2018-11']

In [ ]:
df['2018-01-01': '2018-06-30']

In [ ]:
df = df.drop(columns=['weekday_name', 'weekday_number'])
df

### How about changing the period of our data?

#### Downsampling: Reducing Frequency

In [ ]:
by_month = df.asfreq('M')
by_month.head()

In [ ]:
by_month.close.plot()

##### What if I want the first day of the month?

[Pandas asfreq Offset Aliases](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)

[Pandas asfreq Anchored Offsets](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#anchored-offsets)

In [ ]:
by_month_first_day = df.asfreq('MS')
by_month_first_day.head()

In [ ]:
by_month.close.plot(label='Last Day of Month Frequency')
by_month_first_day.close.plot(label='First Day of Month Frequency')
plt.legend()
plt.show()

#### Upsampling: Increasing Frequency

In [ ]:
by_half_hour = df.asfreq('30T')
by_half_hour.head(10)

#### As freq allows us to fill in the NaNs with two different methods

In [ ]:
by_half_hour.assign(
    ffill=lambda df: df.volume.ffill(),
    bfill=lambda df: df.volume.bfill()
).head()

In [ ]:
by_half_hour = df.asfreq('30T', method='ffill')
by_half_hour.head()

### What about Resampling (Aggregating over time)
   
    * This is similiar to groupby but for dates

In [ ]:
daily_df = df.resample('D').mean()
daily_df

In [ ]:
daily_df.close.plot(label='Daily')
df.resample('W').mean().close.plot(label='Weekly')
df.resample('M').mean().close.plot(label='Monthly')
df.resample('Y').mean().close.plot(label='Yearly')
plt.legend()
plt.show()

### Rolling Average/Windows

In [ ]:
daily_df.rolling(5).mean().head(10)

In [ ]:
df.close.plot(label='Original Granularity')
df.resample('W').mean().close.rolling(4).mean().plot(label='4 week average')
df.resample('W').mean().close.rolling(12).mean().plot(label='12 week average')
plt.legend()
plt.show()

### How about Lagging or Leading the data?

* `.shift`: move the data backwards and forwards by a given amount
* `.diff`: find the difference with the previous observation (or a specified further back observation)

In [ ]:
daily_df = daily_df[['volume', 'close']]
daily_df

In [ ]:
daily_df['shift_by_one'] = daily_df.close.shift(1)
daily_df

In [ ]:
daily_df.close.shift(-1)

In [ ]:
daily_df['diff(1)'] = daily_df.close.diff(1)
daily_df['other_diff'] = daily_df.close - daily_df['shift_by_one']
daily_df

### Let's use shift to answer a question: What is the yearly rate of return or return on investment (ROI)?

### Incoming math warning!

![Math gif](https://media4.giphy.com/media/DHqth0hVQoIzS/giphy.gif?cid=ecf05e47ciwxjpj7j3mkqv6dxplynda0k44lru3atznbajs1&rid=giphy.gif&ct=g)

#### How do we calculate ROI?

`yearly_rate_return` = (`where_we_are_today` - `where_we_were_a_year_ago`  ) / `where_we_were_a_year_ago`

#### Simpler Example

If I buy a rubber ducky for 200 dollars and sell it for 100 dollars after one year, what is my ROI?

In [ ]:
where_we_were_a_year_ago = 200
where_we_are_today = 100
((where_we_are_today - where_we_were_a_year_ago) / where_we_were_a_year_ago) * 100

#### Let's apply the same process to our data

In [ ]:
where_we_were_a_year_ago = daily_df.close.shift(365)
where_we_were_a_year_ago

In [ ]:
daily_df['a_year_ago'] = where_we_were_a_year_ago
daily_df

In [ ]:
daily_df.loc['2019-03-14']

In [ ]:
where_we_were_a_year_ago = daily_df.close.shift(365)
where_we_are_today = daily_df.close

roi_series = ((where_we_are_today - where_we_were_a_year_ago) / where_we_were_a_year_ago) * 100
roi_series

#### And now let's graph it out

In [ ]:
roi_series.dropna().plot()
plt.ylabel('% Return on Investment')
plt.xlabel('Date')
plt.title('1-Year Rate of return on ETH')

In [ ]:
daily_df.close.plot()

### Time Zones

#### Timestamp with no timezone

In [ ]:
t = pd.Timestamp('2020-04-14 10:25:25')
t

#### Specifying the timezone

In [ ]:
t1 = pd.Timestamp('2020-04-14 10:25:25', tz='Us/Central')
t1

#### Convert to a different timezone

In [ ]:
t1 = t1.tz_convert(tz='Asia/tokyo')
t1

### Timedelta

In [ ]:
t

In [ ]:
t + pd.Timedelta('1d')

#### What if I need the time right `now`?

In [ ]:
now = pd.Timestamp.now()
now

#### Now let's to some time math!

In [ ]:
time_since_t = now - t
time_since_t

### What if we just want to return the days?

In [ ]:
pd.Timedelta('1d')

In [ ]:
time_since_t // pd.Timedelta('1d')

#### Simpler way

In [ ]:
time_since_t.days

#### but if we want total seconds then we need to do the math

In [ ]:
days_in_seconds = time_since_t.days * 24 * 60 * 60

seconds_in_timepart = time_since_t.seconds

days_in_seconds + seconds_in_timepart

#### So know timedelta math becomes the simplier way

In [ ]:
time_since_t // pd.Timedelta('1s')

#### Takeaway: Use whichever method gets you answer for you